<a href="https://colab.research.google.com/github/ANUJGAUR/Data-Extraction/blob/main/DataExtraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
import requests
import pandas as pd 
from bs4 import BeautifulSoup
import string
import spacy
import re
import bs4 as bfs
import nltk
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from textblob import TextBlob
import csv
import numpy as np

In [35]:
df = pd.read_excel('/content/drive/MyDrive/Copy of Input.xlsx')
df.head()

,URL_ID,URL
0,37.0,https://insights.blackcoffer.com/ai-in-healthc...
1,38.0,https://insights.blackcoffer.com/what-if-the-c...
2,39.0,https://insights.blackcoffer.com/what-jobs-wil...
3,40.0,https://insights.blackcoffer.com/will-machine-...
4,41.0,https://insights.blackcoffer.com/will-ai-repla...


In [36]:
li = [url for url in df['URL']]
li

['https://insights.blackcoffer.com/ai-in-healthcare-to-improve-patient-outcomes/',
 'https://insights.blackcoffer.com/what-if-the-creation-is-taking-over-the-creator/',
 'https://insights.blackcoffer.com/what-jobs-will-robots-take-from-humans-in-the-future/',
 'https://insights.blackcoffer.com/will-machine-replace-the-human-in-the-future-of-work/',
 'https://insights.blackcoffer.com/will-ai-replace-us-or-work-with-us/',
 'https://insights.blackcoffer.com/man-and-machines-together-machines-are-more-diligent-than-humans-blackcoffe/',
 'https://insights.blackcoffer.com/in-future-or-in-upcoming-years-humans-and-machines-are-going-to-work-together-in-every-field-of-work/',
 'https://insights.blackcoffer.com/how-neural-networks-can-be-applied-in-various-areas-in-the-future/',
 'https://insights.blackcoffer.com/how-machine-learning-will-affect-your-business/',
 'https://insights.blackcoffer.com/deep-learning-impact-on-areas-of-e-learning/',
 'https://insights.blackcoffer.com/how-to-protect-fu

In [49]:
text = []
for url in li:
  text.append(requests.get(url,headers={"User-Agent": "XY"}))

In [50]:
len(text)

114

In [51]:
for i in range(len(text)):
  text[i] = bfs.BeautifulSoup(text[i].content,'html.parser')

In [52]:
articles = []
for text in text:
  headers = {"User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:60.0) Gecko/20100101 Firefox/60.0"}
  page = requests.get(url, headers=headers)
  soup = BeautifulSoup(page.content, 'html.parser')
  articles.append(soup.find(attrs= {"class":"td-post-content"}).text)

In [54]:
for i in range(len(articles)):
  articles[i]= articles[i].replace('\n','')

In [55]:
with open("/content/drive/MyDrive/Copy of StopWords_Auditor.txt","r",encoding = "ISO-8859-1") as aud:
    audwords = aud.read().split("\n")

In [56]:
with open("/content/drive/MyDrive/Copy of StopWords_Currencies.txt","r",encoding = "ISO-8859-1") as cur:
    curwords = cur.read().split("\n")

In [57]:
with open("/content/drive/MyDrive/Copy of StopWords_DatesandNumbers.txt","r",encoding = "ISO-8859-1") as num:
    numwords = num.read().split("\n")

In [58]:
with open("/content/drive/MyDrive/Copy of StopWords_Generic.txt","r",encoding = "ISO-8859-1") as gen:
    genwords = gen.read().split("\n")

In [59]:
with open("/content/drive/MyDrive/Copy of StopWords_GenericLong.txt","r",encoding = "ISO-8859-1") as lon:
    lonwords = lon.read().split("\n")

In [60]:
with open("/content/drive/MyDrive/Copy of StopWords_Geographic.txt","r",encoding = "ISO-8859-1") as geo:
    geowords = geo.read().split("\n")

In [61]:
with open("/content/drive/MyDrive/Copy of StopWords_Names.txt","r",encoding = "ISO-8859-1") as nam:
    namwords = nam.read().split("\n")

In [68]:
stop_words = audwords+curwords+numwords+genwords+lonwords+geowords+namwords

In [64]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [65]:
sentences = []
for article in articles:
  sentences.append(len(sent_tokenize(article)))

In [66]:
cleaned_articles = [' ']*len(articles)

In [69]:
for i in range(len(articles)):
  for w in stop_words:
    cleaned_articles[i]= articles[i].replace(' '+w+' ',' ').replace('?','').replace('.','').replace(',','').replace('!','')

In [70]:
words = []
for article in articles:
  words.append(len(word_tokenize(article)))

In [71]:
words_cleaned = []
for article in cleaned_articles:
  words_cleaned.append(len(word_tokenize(article)))

In [72]:
with open("/content/drive/MyDrive/Copy of positive-words.txt","r") as pos:
    poswords = pos.read().split("\n")  

In [73]:
pos_count = " ".join ([w for w in words_cleaned if w in poswords])
pos_count=pos_count.split(" ")

In [74]:
positive_score = [0]*len(articles)
for i in range(len(articles)):
  for word in pos_count:
    for letter in cleaned_articles[i].lower().split(' '):
      if letter==word:
        positive_score[i]+=1

In [75]:
with open("/content/drive/MyDrive/Copy of negative-words.txt","r",encoding = "ISO-8859-1") as neg:
    negwords = neg.read().split("\n")

In [77]:
neg_count = " ".join ([w for w in words_cleaned if w in negwords])
neg_count=neg_count.split(" ")

In [78]:
negative_score = [0]*len(articles)
for i in range(len(articles)):
  for word in neg_count:
    for letter in cleaned_articles[i].upper().split(' '):
      if letter==word:
        negative_score[i]+=1

In [79]:
words_cleaned = np.array(words_cleaned)
sentences = np.array(sentences)

In [80]:
df['POSITIVE SCORE'] = positive_score
df['NEGATIVE SCORE'] = negative_score

In [81]:
df['POLARITY SCORE'] = (df['POSITIVE SCORE']-df['NEGATIVE SCORE'])/ ((df['POSITIVE SCORE'] +df['NEGATIVE SCORE']) + 0.000001)

In [82]:
df['SUBJECTIVITY SCORE'] = (df['POSITIVE SCORE'] + df['NEGATIVE SCORE'])/( (words_cleaned) + 0.000001)

In [83]:
df['AVG SENTENCE LENGTH'] = np.array(words)/np.array(sentences)

In [84]:
complex_words = []
sylabble_counts = []

In [85]:
for article in articles:
  sylabble_count=0
  d=article.split()
  ans=0
  for word in d:
    count=0
    for i in range(len(word)):
      if(word[i]=='a' or word[i]=='e' or word[i] =='i' or word[i] == 'o' or word[i] == 'u'):
           count+=1
      if(i==len(word)-2 and (word[i]=='e' and word[i+1]=='d')):
        count-=1;
      if(i==len(word)-2 and (word[i]=='e' and word[i]=='s')):
        count-=1;
    sylabble_count+=count    
    if(count>2):
        ans+=1
  sylabble_counts.append(sylabble_count)
  complex_words.append(ans)   

In [86]:
df['PERCENTAGE OF COMPLEX WORDS'] = np.array(complex_words)/np.array(words)

In [87]:
df['FOG INDEX'] = 0.4 * (df['AVG SENTENCE LENGTH'] + df['PERCENTAGE OF COMPLEX WORDS'])

In [88]:
df['AVG NUMBER OF WORDS PER SENTENCES'] = df['AVG SENTENCE LENGTH']

In [89]:
df['COMPLEX WORD COUNT'] = complex_words

In [90]:
df['WORD COUNT'] = words

In [91]:
df['SYLLABLE PER WORD'] = np.array(sylabble_counts)/np.array(words)

In [92]:
total_characters = []
for article in articles:
  characters = 0
  for word in article.split():
    characters+=len(word)
  total_characters.append(characters)  

In [93]:
personal_nouns = []
personal_noun =['I', 'we','my', 'ours','and' 'us','My','We','Ours','Us','And'] 
for article in articles:
  ans=0
  for word in article:
    if word in personal_noun:
      ans+=1
  personal_nouns.append(ans)

In [94]:
df['PERSONAL PRONOUN'] = personal_nouns

In [95]:
df['AVG WORD LENGTH'] = np.array(total_characters)/np.array(words)

In [97]:
df.head()

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCES,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUN,AVG WORD LENGTH
0,37.0,https://insights.blackcoffer.com/ai-in-healthc...,0,0,0.0,0.0,18.290323,0.234568,7.409956,18.290323,266,1134,1.758377,8,4.738977
1,38.0,https://insights.blackcoffer.com/what-if-the-c...,0,0,0.0,0.0,18.290323,0.234568,7.409956,18.290323,266,1134,1.758377,8,4.738977
2,39.0,https://insights.blackcoffer.com/what-jobs-wil...,0,0,0.0,0.0,18.290323,0.234568,7.409956,18.290323,266,1134,1.758377,8,4.738977
3,40.0,https://insights.blackcoffer.com/will-machine-...,0,0,0.0,0.0,18.290323,0.234568,7.409956,18.290323,266,1134,1.758377,8,4.738977
4,41.0,https://insights.blackcoffer.com/will-ai-repla...,0,0,0.0,0.0,18.290323,0.234568,7.409956,18.290323,266,1134,1.758377,8,4.738977


In [98]:
df.to_csv('Output Data Structure.csv')

In [99]:
articles

['Big DataTo begin with I shall first like to explain what big data is and why it has become so important in our lives. Big data is simply data, but with a huge size. Data that is not just voluminous but also growing exponentially with time. Such data sets are so large and complex that it is not possible to store or process them using traditional data management tools. So, how huge can this data be? Social media sites like Facebook generate more than 500 terabytes of new data every day in the form of photos, video uploads, text messages, etc. A single\xa0Jet engine\xa0can generate\xa0more than 10 terabytes\xa0of data in\xa030 minutes\xa0of flight time. With many thousand flights per day, generation of data reaches up to many\xa0petabytes. Data contained in these sets are not always structured. It can be semi-structured or even unstructured. When such is the size and dimension of data we can well imagine how complicated it must be to process and analyze it. Therefore, modern methods of 

In [100]:
with open('URL_ID .txt', 'w') as f:
    f.write('articles')